# Влияние витамина C на рост зубов морских свинок
В эксперименте изучалось влияние витамина C на рост [одонтобластов](https://en.wikipedia.org/wiki/Odontoblast) морских свинок.



Рассматривалось три дозы витамина (0.5, 1 и 2 мг) и два способа его приёма (апельсиновый сок и в чистом виде (== аскорбиновая кислота)). В каждой из 6 групп по уровням двух факторов были произведены измерения для 10 морских свинок.

## Загрузим данные и посмотрим на них

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as st
import pandas as pd
import seaborn as sns

In [ ]:
data = pd.read_csv('ToothGrowth.csv')
data.head()

In [ ]:
data = data.drop(columns='Unnamed: 0')
data.head()

In [ ]:
data.supp.unique()

In [ ]:
_ = sns.boxplot(x='dose', y='len', data=data)

In [ ]:
_ = sns.boxplot(x='supp', y='len', data=data)

## Двухфакторный дисперсионный анализ

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

Без учета межфакторного взаимодействия:

In [ ]:
lm = ols('len ~ C(supp) + C(dose)',data=data).fit()
table = sm.stats.anova_lm(lm)
table

С учетом межфакторного взаимодействия:

In [ ]:
lm = ols('len ~ C(supp) * C(dose)', data=data).fit()
table = sm.stats.anova_lm(lm, typ=1)
table

### Межфакторное взаимодействие:

$$X_{ijk} = \mu + \alpha_i + \beta_j + \gamma_{ij} + \varepsilon_{ijk},$$
$$i = 1, ..., K_1,\ j = 1, ..., K_2,\ k = 1, ..., n.$$

- $\mu$ -- общее среднее значение признака,
- $\alpha_i$ -- воздействие уровня $i$ фактора $f_1$,
- $\beta_j$ -- воздействие уровня $j$ фактора $f_2$,
- $\gamma_{ij}$ -- дополнительное воздействие комбинации уровней $i$ и $j$ факторов $f_1$ и $f_2$,
- $\varepsilon_{ijk}$ -- случайные независимые одинаково распределенные ошибки

`sns.pointplot`: 
- точки -- средние значения исследуемого параметра по каждой из групп (значению категориальной переменной)
- 95%-й доверительный интервал

In [ ]:
_ = sns.pointplot(x="dose", y="len", data=data, hue='supp')
plt.show()
_ = sns.pointplot(x="supp", y="len", data=data, hue='dose')

Взаимодействие между факторами можно наблюдать, когда результат от воздействия одного фактора на зависимую переменную не одинаков при разных значениях другого фактора.

## Взаимодействие значимо, поэтому разделим выборку по способу приёма витамина.

In [ ]:
OJ = data[data['supp']=='OJ']
VC = data[data['supp']=='VC']
VC.shape, OJ.shape

### Для морских свинок, принимающих аскорбиновую кислоту:

In [ ]:
_ = sns.boxplot(x='dose', y='len', data=VC)

In [ ]:
lm = ols('len ~ C(dose)',data=VC).fit()
table = sm.stats.anova_lm(lm)
table

Для морских свинок, принимающих аскорбиновую кислоту влияние разных уровней фактора дозы неодинаково

#### Критерий Краскела-Уоллиса о равенстве средних всех групп

In [ ]:
st.kruskal(*VC.groupby('dose').groups.values())

#### Исследуем попарные различия в воздействии на целевой признак между всеми уровнями фактора дозы

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison
mc = MultiComparison(VC['len'], VC['dose'])
result = mc.tukeyhsd()
 
print(result)

Значимые различия во влиянии на целевую переменную есть между каждой парой групп

### Для морских свинок, принимающих апельсиновый сок:

In [ ]:
_ = sns.boxplot(x='dose', y='len', data=OJ)

In [ ]:
lm = # ваш код
table = sm.stats.anova_lm(lm)
table

Отвергаем гипотезу о том, что каждый уровень фактора дозы вносит одинаковый вклад в целевую переменную

#### Исследуем попарные различия в воздействии на целевой признак между всеми уровнями фактора дозы

In [ ]:
# tukeyhsd - ваш код

Значимые отличия в воздействии на целевой признак отсутствуют только между 1 и 2

In [ ]:
_ = sns.pointplot(x="dose", y="len", data=OJ)
plt.title('Pointplot for orange juice')

## Итого

- для морских свинок, принимающих аскорбиновую кислоту, различия между размером дозы значимы всегда
- для морских свинок, принимающих апельсиновый сок, различия между размером дозы в 1 и 2 мг не значимо

**Crampton E. W. (1947) The growth of the odontoblast of the incisor teeth as a criterion of vitamin C intake of the guinea pig. The Journal of Nutrition 33(5): 491–504.**